In [1]:
import numpy as np
from aocd import get_data, submit
from operator import itemgetter
import re
import collections

In [2]:
example_data = "root: pppw + sjmn\ndbpl: 5\ncczh: sllz + lgvd\nzczc: 2\nptdq: humn - dvpt\ndvpt: 3\nlfqf: 4\nhumn: 5\nljgn: 2\nsjmn: drzm * dbpl\nsllz: 4\npppw: cczh / lfqf\nlgvd: ljgn * ptdq\ndrzm: hmdt - zczc\nhmdt: 32"
real_data = get_data(day=21, year=2022)

In [3]:
def run_operation(value1, operation, value2):
    if operation == "+":
        return value1 + value2
    elif operation == "-":
        return value1 - value2
    elif operation == "*":
        return value1 * value2
    elif operation == "/":
        return  value1 / value2
    else:
        raise ValueError("Unknown operation!")

def revert_operation(value, index_known, result, operation):
    if operation == "+":
        return result-value
    elif operation == "-":
        return result + value if index_known==1 else value - result
    elif operation == "*":
        return result / value
    elif operation == "/":
        return result * value if index_known==1 else value / result
    elif operation == "=":
        return value
    else:
        raise ValueError("Unknown operation!")

class Monkey:
    def __init__(self, dependency_names, operation, resolved, is_me, is_root):
        self.dependency_names = dependency_names
        self.operation = operation
        self.resolved = resolved
        self.dependencies = []
        self.depends_on_me = False
        self.parent = None
        self.is_me = is_me
        self.is_root = is_root

    def resolve(self):
        if self.resolved is None:
            self.resolved = run_operation(self.dependencies[0].resolve(), self.operation, self.dependencies[1].resolve())

        return self.resolved

In [4]:
pattern_operation = re.compile(r'^([a-z]{4}): ([a-z]{4}) ([\+\-\*\/]) ([a-z]{4})$')
pattern_resolved = re.compile(r'^([a-z]{4}): (\d+)$')

monkeys = dict()

for line in real_data.splitlines():
    match = pattern_resolved.match(line)
    if match:
        name, value = match.groups()
        monkeys[name] = Monkey([], None, int(value), name == "humn", name == "root")
    else:
        name, dependency1, operation, depedency2 = pattern_operation.match(line).groups()
        monkeys[name] = Monkey([dependency1, depedency2], operation, None, name == "humn", name == "root")

for monkey_name in monkeys:
    monkey = monkeys[monkey_name]
    if len(monkey.dependency_names) >= 0:
        for dependent_monkey in monkey.dependency_names:
            monkey.dependencies.append(monkeys[dependent_monkey])
            monkeys[dependent_monkey].parent = monkey

current = monkeys["humn"]
current.depends_on_me = True
while current.parent is not None:
    current = current.parent
    current.depends_on_me = True

In [5]:
root = monkeys["root"]
print(f"Value of root: {int(root.resolve())}")

Value of root: 51928383302238


In [6]:
def resolve_me(monkey, result):
    if monkey.is_me:
        return result

    operation = "=" if monkey.is_root else monkey.operation

    if monkey.dependencies[0].depends_on_me and not monkey.dependencies[1].depends_on_me:
        value = monkey.dependencies[1].resolve()
        next_result = revert_operation(value, 1, result, operation)
        return resolve_me(monkey.dependencies[0], next_result)
    elif monkey.dependencies[1].depends_on_me and not monkey.dependencies[0].depends_on_me:
        value = monkey.dependencies[0].resolve()
        next_result = revert_operation(value, 0, result, operation)
        return resolve_me(monkey.dependencies[1], next_result)
    else:
        raise ValueError("Cannot resolve" + str(monkey.dependency_names))

print(f"My number: {int(resolve_me(root, None))}")

My number: 3305669217840
